# Code to turn FBA data and proteomics into estimates for Kapps
## Written by: Wheaton Schroeder
### Latest version: 01/01/2024

## Workflow taken from supplemental file of Hoang and Maranas. Metabolic Engineering, 2023.
#### Rules used taken from Dinh and Maranas, 2023
##### 1. Enzymes that catalyze multiple reactions are assumed to have the same Kapp for all reactions. This is calculated by summing flux rates and dividing by that enzyme's concentration.
###### assumption: absolute sum of flux rates
##### 2. In "and" relationships, the lowest abundant protein divided by its stoichiometry is used to set the enzyme concentration
##### 3. Any reaction with a "0" rate under pFBA will be assigned the average Kapp value so that it can still hold flux and still have some enzyme limitation

#### Make imports

In [1]:
import re
import pandas as pd
import numpy as np
import cobra
import shutil

# Input datasets

In [2]:
#set the default folder
def_in = "input_base"
def_out = "model_base"

#### read the proteomics file

In [3]:
proteomics = pd.read_excel("./"+def_in+"/get_kapp/simplified_protein_measurements.xlsx")

#change locus tag to index
proteomics = proteomics.set_index("Locus Tags")

#for debugging
proteomics

,No,Fasta headers,Mass (Da),iBAQ (log2),g/gDCW
Locus Tags,,,,,
Clo1313_2479,1,CP002416.1_prot_ADU75492.1_2392[locus_tag=Clo1...,249175.045,13.597222,6.686032e-08
Clo1313_1249,2,CP002416.1_prot_ADU74312.1_1212[locus_tag=Clo1...,81307.705,14.367733,4.084791e-08
Clo1313_1075,3,CP002416.1_prot_ADU74139.1_1039[locus_tag=Clo1...,25442.325,14.999780,2.138299e-08
Clo1313_0956,4,CP002416.1_prot_ADU74024.1_924[locus_tag=Clo13...,46594.370,15.077650,4.172350e-08
Clo1313_0147,5,CP002416.1_prot_ADU73243.1_143[locus_tag=Clo13...,43346.730,15.084033,3.901762e-08
...,...,...,...,...,...
Clo1313_0023,1968,CP002416.1_prot_ADU73123.1_23[locus_tag=Clo131...,34648.880,30.327317,7.892410e-03
Clo1313_1797,1969,CP002416.1_prot_ADU74853.1_1753[locus_tag=Clo1...,28586.640,30.341843,6.589397e-03
Clo1313_3011,1970,CP002416.1_prot_ADU75989.1_2889[locus_tag=Clo1...,113156.140,30.809773,3.825249e-02


#### Read in the enzyme stoich file to get the GPR for each reaction and each reaction name

In [4]:
rxn_data = pd.read_excel("./"+def_in+"/ENZYME_stoich_curation_ctherm.xlsx")

#change locus tag to index
rxn_data = rxn_data.set_index("id")

#for debugging
rxn_data

,rxn_src,enz,gpr,protein_stoich,subunit_comments,status,MW (g/mmol)
id,,,,,,,
RXN-13PPDH_FWD-BDH,13PPDH,BDH,Clo1313_2130,Clo1313_2130:1,,protStoichAsgnAuto,43.33800
RXN-13PPDH_FWD-gbsB_1798,13PPDH,gbsB_1798,Clo1313_1798,Clo1313_1798:1,,protStoichAsgnAuto,96.85200
RXN-13PPDH_FWD-gbsB_1827,13PPDH,gbsB_1827,Clo1313_1827,Clo1313_1827:1,,protStoichAsgnAuto,42.48978
RXN-2D3DGLNR_FWD-kduD,2D3DGLNR,kduD,Clo1313_0815,Clo1313_0815:1,,protStoichAsgnAuto,27.91363
RXN-2D3DGLNR_REV-kduD,2D3DGLNR,kduD,Clo1313_0815,Clo1313_0815:1,,protStoichAsgnAuto,27.91363
...,...,...,...,...,...,...,...
RXN-Zn2divalent_FWD-SPONT,Zn2divalent,SPONT,SPONT,zeroCost,NaN,protStoichAsgnManual,89.80625
RXN-Zn2metal_FWD-SPONT,Zn2metal,SPONT,SPONT,zeroCost,,protStoichAsgnAuto,0.00000
RXN-ZNabc_FWD-znuABC,ZNabc,znuABC,Clo1313_1688 and Clo1313_1689 and Clo1313_1690,"Clo1313_1688:1,Clo1313_1689:1,Clo1313_1690:1",,protStoichAsgnAuto,0.00000


#### read in the protein stoichiometry to get MW

In [5]:
prot_data = pd.read_excel("./"+def_in+"/PROTEIN_stoich_curation_ctherm.xlsx")

#change locus tag to index
prot_data = prot_data.set_index("id")

#for debugging
prot_data

,id_unknown_src,gene_src,name,uniprot,subloc_assigned,cofactor_stoich,cofactor_comment,MW (g/mmol),sequence,status
id,,,,,,,,,,
Clo1313_0011,ADU73111,Clo1313_0011,serS,A3DI04,c,NaN,NaN,48.88727,MLDIKLIRSNPEILKKALQKRKDNFDVNGLLSLDEKRRKTLVELEQ...,NaN
Clo1313_0020,ADU73120,Clo1313_0020,ADU73120,A3DI13,c,NaN,NaN,21.37628,MGKVVEIRWHGRGGQGAKTASLLLADAAFNTGKYIQGFPEYGPERM...,NaN
Clo1313_0021,ADU73121,Clo1313_0021,ADU73121,A3DI14,c,4fe4s_c:1,NaN,11.33791,MSKELRDVKPDVTWKEITSGGVIDSPGNAHLFKTGDWRSMKPVWNE...,NaN
Clo1313_0022,ADU73122,Clo1313_0022,ADU73122,A3DI15,c,NaN,NaN,43.82318,MGIRERLSGNEATAIAMRQINPDVVAAFPITPSTEIPQYFSSYVAD...,NaN
Clo1313_0023,ADU73123,Clo1313_0023,ADU73123,A3DI16,c,NaN,NaN,34.94988,MAYNLKEVAKKPERLTGGHRMCAGCGAPIVVRQVLKALKPEDHAVI...,NaN
...,...,...,...,...,...,...,...,...,...,...
Clo1313_0693,ADU73772,Clo1313_0693,ADU73772,no_entry,c,NaN,NaN,92.01549,MRKRKLGFSAVIFITIIAIFGCFLDLRISAAPNEYKFDFGAGPVEP...,added cellulosome machinery (unknown/other fun...
Clo1313_0950,ADU74018,Clo1313_0950,ADU74018,no_entry,c,NaN,NaN,69.27882,MRKKKRLISLLLAVFIAVACLPAGIARADKASSIELKFDRNKGEVG...,added cellulosome machinery (s-layer homology ...
Clo1313_0630,ADU73710,Clo1313_0630,ADU73710,no_entry,c,NaN,NaN,48.74320,MKRIKRILAVLTIFALLATINAFTFVSLAQTNTIEIIIGNVKARPG...,added cellulosome machinery (s-layer homology ...


# Calculate steady-state translation fluxes v^exp_pro

In [6]:
#for each protein, multiply growth rate by protein concentration to get synthesis rate
growth = 0.4

#stores required rate of protein synthesis
v_exp_pro = dict()

#store concentration
prot_mmolgDCW = dict()

#stores list of proteins
prot_list = proteomics.index.to_list()

for prot in prot_list:

  #get g protein per gDCW
  prot_ggDW_temp = proteomics.loc[prot, "g/gDCW"]
  
  #get protein MW
  prot_MW_temp = proteomics.loc[prot, "Mass (Da)"]

  #convert to mmol/gDCW of protein
  prot_mmolgDCW[prot] = (prot_ggDW_temp / prot_MW_temp) * 1000

  #get the required rate of protein synthesis
  v_exp_pro[prot] = prot_mmolgDCW[prot] * growth
    
v_exp_pro

{'Clo1313_2479': 1.0733067845939916e-10,
 'Clo1313_1249': 2.0095467965088667e-10,
 'Clo1313_1075': 3.361798541344463e-10,
 'Clo1313_0956': 3.5818494460116365e-10,
 'Clo1313_0147': 3.6005134724870767e-10,
 'Clo1313_0842': 4.825119637120581e-10,
 'Clo1313_2667': 4.90408100764004e-10,
 'Clo1313_1425': 5.567602379270293e-10,
 'Clo1313_0351': 5.779534222048227e-10,
 'Clo1313_2858': 5.902387096188006e-10,
 'Clo1313_0639': 5.905183676861789e-10,
 'Clo1313_2965': 6.825082202409582e-10,
 'Clo1313_0352': 7.704045519585974e-10,
 'Clo1313_2541': 9.394398590417208e-10,
 'Clo1313_2648': 9.822116983211954e-10,
 'Clo1313_0122': 1.0361288661405855e-09,
 'Clo1313_2583': 1.090923622301006e-09,
 'Clo1313_0015': 1.1609106313191905e-09,
 'Clo1313_1930': 1.1826985742013811e-09,
 'Clo1313_0042': 1.2872859323991644e-09,
 'Clo1313_2713': 1.2904971364478684e-09,
 'Clo1313_0735': 1.2923831709343819e-09,
 'Clo1313_1939': 1.2993365603220644e-09,
 'Clo1313_2749': 1.4599952745641584e-09,
 'Clo1313_0234': 1.4733910001

# Calculate enzyme concentration [Enz] accounting for protein subunit stoichiometry (coefficients S_pro,enz)

#### This block identifies which protein concentrations should be associated with each enzyme, assigns maps from reactions to enzymes, and flag enzymes which lack measurements

In [7]:
#store RBA model names
rxn_names = rxn_data.index.to_list()

#stores if reaction has at least one measured protein abundance. If not, this reaction will be treated as unmeasured.
rxn_prot_meas = dict()

#build dictionary for protein stoichiometry for each reaction
enz_prot_stoic = dict()

#build dictionary for enzyme names
rxn_enz_name = dict()

#list of enzymes, list and set used to make that list non-redunant
enz_list = list(set(rxn_data.enz.to_list()))

#enzyme-keyed and protein-keyed concentration of enzyme components
enz_comp_conc = dict()

#base name to enzyme name
base_to_enz = dict()

#stores if an enzyme has at least one enzyme that is measured
enz_prot_meas = dict()

#sub-dictionaries need to be initialized for components and concentrations
for enz in enz_list:

  #initialize nested dictionaries for component concentration, reaction rates, and protein stoichiometry
  enz_comp_conc[enz] = dict()
  enz_prot_stoic[enz] = dict()

  #default enzymes to being unmeasured
  enz_prot_meas[enz] = False

#keep a count of unmeasured genes
unmeasured = list()

#keep a count of measured genes
measured = list()

#keep a count of zero cost results
zero_cost = dict()

#keep a list of genes
all_genes = list()

#keep a list of reactions where no proteomics measurement is present
no_meas = list()

for rxn in rxn_names:

  #get the base reaction name, makes fluxes easier to look up
  #get the base reaction name by removing textual framework
  base_name = rxn.split("RXN-")

  base_name2 = re.split("_(FWD|REV)",base_name[1])

  stoich_str_temp = rxn_data.loc[rxn, 'protein_stoich']

  #default to the reaction proteins being unmeasured
  rxn_prot_meas[rxn] = False
  
  #for debugging
  #print("rxn: ",rxn)
  #print("protein_stoich: ",stoich_str_temp)

  #split the protein stoichiometry into individual genes
  stoich_list_temp = stoich_str_temp.split(",")

  #for debugging
  #print('rxn: ',rxn,"\tstoich_list_temp: ",stoich_list_temp)

  rxn_enz_name[rxn] = rxn_data.loc[rxn, 'enz']

  if base_name2[0] in base_to_enz:

    if not rxn_data.loc[rxn, 'enz'] in base_to_enz[base_name2[0]]:

      base_to_enz[base_name2[0]].append(rxn_data.loc[rxn, 'enz'])

  else:

    base_to_enz[base_name2[0]] = list()
    base_to_enz[base_name2[0]].append(rxn_data.loc[rxn, 'enz'])
  
  if(stoich_list_temp[0] == 'zeroCost'):

    #we can't get a kapp estimate it is a zero cost reaction
    zero_cost[rxn] = True

  else:

    #is not a zero-cost reaction
    zero_cost[rxn] = False

    #for each gene
    for gene in stoich_list_temp:

      gene_temp, stoich_temp = gene.split(":")

      if gene_temp in proteomics.index:

        #if here, then it is measured

        #that means at least one protein is measured
        rxn_prot_meas[rxn] = True

        #at least one protien in the enzyme is measured
        enz_prot_meas[rxn_data.loc[rxn, 'enz']] = True

        #get the measured amount, based on log 2 values, as these likely store more precision
        ggDW_temp = prot_mmolgDCW[gene_temp]
        
        #save the stoichiometry and concentration
        enz_prot_stoic[rxn_data.loc[rxn, 'enz']][gene_temp] = float(stoich_temp)

        #save enzyme component concentrations if not already there
        if not (gene_temp in enz_comp_conc[rxn_enz_name[rxn]]):

          #save the concentration
          enz_comp_conc[rxn_enz_name[rxn]][gene_temp] = ggDW_temp

        #for debugging 
        if gene_temp == "Clo1313_1502":

          print("enzyme: ",rxn_enz_name[rxn],"\tgene: ",gene_temp,"\tstoich: ",stoich_temp,"\tconcentration: ",ggDW_temp," mmol/gDCW")

        #update list and counts only if gene not already in the gene list
        if gene_temp not in all_genes:

          measured.append(gene_temp)
          all_genes.append(gene_temp)

      else:

        #otherwise, unmeasured

        #for debugging
        #print("gene: ",gene_temp,"\tstoich: ",stoich_temp,"\tis unmeasured")

        #update list and counts only if gene not already in the gene list
        if gene_temp not in all_genes:

          unmeasured.append(gene_temp)
          all_genes.append(gene_temp)

  #for debugging for the reaction, report the enzyme concentration bound that we will use
  #print("reaction: ",rxn,"\tenzyme: ",rxn_enz_name[rxn],"\tat least one protein measured? ",rxn_prot_meas[rxn])

  #do something if a bound wasn't set
  if not rxn_prot_meas[rxn]:

    no_meas.append(rxn)


enzyme:  iscSU 	gene:  Clo1313_1502 	stoich:  2 	concentration:  2.0027322043846746e-05  mmol/gDCW


#### determine enzyme concentration, flag reactions as active or inactive.

In [8]:
# for "and" relations, where there are multiple proteins making up a heteromer, use the smallest to calcualte enzyme concentration
#note that "or" relations are already separated out, wherever tehre are multiple

#create a dictionary to store enzyme concentration
enz_conc = dict()

#build a dictionary to store if an enzyme is active
enz_active = dict()

#for each enzyme
for enz in enz_list:

  #only bother if the protein is measured
  if enz_prot_meas[enz]:

    #for debugging
    #print("enzyme: ",enz,"\tamount: ",enz_comp_conc[enz])

    #only bother if has at least one protein in the enzyme
    #should be the same as if measured, but 
    if(len(enz_comp_conc[enz]) >= 1):

      #stores the list of adjusted concentrations
      stoic_conc = list()

      #for each protein in the stoichiometry of the enzyme
      for prot in enz_comp_conc[enz]:

        #calculate concentration
        comp_temp = enz_comp_conc[enz][prot]
        stoic_temp = enz_prot_stoic[enz][prot]
        stoic_conc_temp = enz_comp_conc[enz][prot] / enz_prot_stoic[enz][prot]

        stoic_conc.append(stoic_conc_temp)
      
      enz_conc[enz] = min(stoic_conc)

    else:

      #do nothing, no proteins
      pass

    if(enz_conc[enz] < 1E-10):

      enz_active[enz] = False
    
    else:

      enz_active[enz] = True

    #for debugging
    #print("\"and\"-based concentration: ",enz_conc[enz],"\tactive? ",enz_active[enz])
      
  else:

    #if unmeasured, assume inactive
    enz_active[enz] = False

#don't flag spontaneous reactions as inactive
enz_active['SPONT'] = True

#### Take enzyme activity flags, turn them into reaction activity flags

In [9]:
#list of inactive reactions
inactive = list()

for rxn in rxn_names:

  #if the associated enzyme is already deemed inactive, add to the inactive list
  if not enz_active[rxn_enz_name[rxn]]:

    inactive.append(rxn)

print(inactive)
len(inactive)

['RXN-3MOPDC_FWD-UNKNOWN', 'RXN-3OACOAS_FWD-UNKNOWN', 'RXN-4MOPDC_FWD-UNKNOWN', 'RXN-AACPS3MBUT_FWD-UNKNOWN', 'RXN-AACPS3MBUT_REV-UNKNOWN', 'RXN-AACPS3MCR_FWD-UNKNOWN', 'RXN-AACPS3MCR_REV-UNKNOWN', 'RXN-ACabc_FWD-UNKNOWN', 'RXN-ACGNMCT_FWD-ADU73322', 'RXN-ACLDC_FWD-UNKNOWN', 'RXN-ACNAM9PL_FWD-spsE_0230', 'RXN-ACNMCT_FWD-ADU73322', 'RXN-ACNPLYS_FWD-spsE_0230', 'RXN-ACt2_FWD-UNKNOWN', 'RXN-ADCL_FWD-UNKNOWN', 'RXN-ADEabc_FWD-nupOPQ', 'RXN-ADEabc_REV-nupOPQ', 'RXN-ADPCOAL_FWD-grsT', 'RXN-ADSK_FWD-tufA', 'RXN-AKP1_FWD-UNKNOWN', 'RXN-ALAALAr_FWD-pyrAB', 'RXN-ALCD22xi_FWD-cotJA', 'RXN-ALCD22xi2_FWD-cotJA', 'RXN-ALCD23xi2_FWD-cotJA', 'RXN-ALCD3MBOH_FWD-cotJA', 'RXN-ALCD3x_FWD-cotJA', 'RXN-AMAOTr_FWD-bioA', 'RXN-AMAOTr_REV-bioA', 'RXN-AMUAAH_FWD-cwlD', 'RXN-AMUAAH_FWD-cwlJ', 'RXN-AMUAAH_FWD-lysM', 'RXN-AMUAAH_FWD-sleB_0046', 'RXN-AMUAAH_FWD-sleB_2624', 'RXN-AMUAAH_FWD-SLH', 'RXN-AMUAAH_FWD-yqiI_1198', 'RXN-AOXSr_FWD-bioF', 'RXN-AOXSr_REV-bioF', 'RXN-AOXSr2_FWD-bioF', 'RXN-ASNS1_FWD-ADU74738', '

269

# Make Reports

In [10]:
#make a report  to get some idea as to 
unmeas_frac = len(unmeasured)/len(all_genes)
meas_frac = len(measured)/len(all_genes)
no_meas_frac = len(no_meas) / len(rxn_names)
inactive_frac = len(inactive) / len(rxn_names)

print("fraction genes without measurements: ",unmeas_frac)
print("fraction genes with measurements: ",meas_frac)
print("fraction reactions without gene measurements: ",no_meas_frac)
print("fraction reactions deemed inactive by enzyme amount: ",inactive_frac)

fraction genes without measurements:  0.16612377850162866
fraction genes with measurements:  0.8338762214983714
fraction reactions without gene measurements:  0.2504307869040781
fraction reactions deemed inactive by enzyme amount:  0.15450890292935096


#### create some report files, as well as files for GAMS to use

In [19]:
#list of unmeasured genes
unmeas_gene_f = open('./'+def_in+'/scripts_to_make_inputs/script_outputs/unmeasured_genes.txt', 'w')

for gene in unmeasured:

  unmeas_gene_f.write(gene+"\n")

unmeas_gene_f.close()

#list of measured genes
meas_gene_f = open('./'+def_in+'/scripts_to_make_inputs/script_outputs/measured_genes.txt', 'w')

for gene in measured:

  meas_gene_f.write(gene+"\n")

meas_gene_f.close()

#list of reactions without a corresponding gene measurement
no_meas_rxn_f = open('./'+def_in+'/scripts_to_make_inputs/script_outputs/no_meas_rxns.txt', 'w')

for rxn in no_meas:

  no_meas_rxn_f.write(rxn+"\n")

no_meas_rxn_f.close()

#list of inactive reactions
#this should be formatted as a GAMS file, will be used as an input
inactive_rxn_f = open('./'+def_out+'/inactive_rxns.txt', 'w')
inactive_rxn_f.write("/\n")

for rxn in inactive:

  inactive_rxn_f.write("'"+rxn+"'\n")

inactive_rxn_f.write("/")
inactive_rxn_f.close()

#previous code for writing protein synthesis rate constraints, now performed in a different file
##write required protein synthesis rates as a GAMS input file
#prot_rate_f = open('./'+def_out+'/prot_rate_exp.txt', 'w')
#prot_rate_f.write("/\n")
#
#prot_rate_rxns_f = open('./'+def_out+'/prot_rate_exp_rxns.txt', 'w')
#prot_rate_rxns_f.write("/\n")
#
##note: only going to enforce the protein synthesis for model protiens, rest will be covered by dummy proteins
#for prot in measured:
#
#
  #prot_rate_rxns_f.write("\'PROSYN-"+prot+"\'\n")
#
  #prot_rate_f.write("\'PROSYN-"+prot+"\'\t"+str(v_exp_pro[prot])+"\n")
#
#
#prot_rate_f.write("/")
#prot_rate_f.close()
#
#prot_rate_rxns_f.write("/")
#prot_rate_rxns_f.close()

#### build input file for per-input carbon yeild 

In [12]:
############################ DEFINE CONSTRAINTS FOR APPLYING MFA DATA ############################

#build dictionaries of upper and lower bounds for fraction of carbon to dedicate to 
#frac_min and frac_max values are minimum and maximum fractions of input carbon destined for a particular output molecule
#the rxns list is to provide a list of those reactions, as a subset of 
carb_frac_min = dict()
carb_frac_max = dict()
carb_frac_rxns = list()

#note: will do the calculations here from the MFA ranges directly.
#MFA ranges need to be divided by the total glucose unit input (600 mmol C/gDWh) to scale production per carbon input
#We won't turn off 

######## ETHANOL ########

#ethanol export
#define bounds
carb_frac_rxns.append("RXN-EXCH_etoh_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_etoh_e_FWD-SPONT"] = (24.7319) / 100
carb_frac_min["RXN-EXCH_etoh_e_FWD-SPONT"] = (20.655) / 100

######## ACETATE ########

#acetate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_ac_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_ac_e_FWD-SPONT"] = (34.3793) / 100
carb_frac_min["RXN-EXCH_ac_e_FWD-SPONT"] = (29.9675) / 100

######## CO2 ########

#CO2 export
#define bounds
carb_frac_rxns.append("RXN-EXCH_co2_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_co2_e_FWD-SPONT"] = (79.6414) / 100
carb_frac_min["RXN-EXCH_co2_e_FWD-SPONT"] = (74.0563) / 100

######## PYRUVATE ########

#CO2 export
#define bounds
carb_frac_rxns.append("RXN-EXCH_pyr_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_pyr_e_FWD-SPONT"] = (6.26) / 100
carb_frac_min["RXN-EXCH_pyr_e_FWD-SPONT"] = (5.1795) / 100

######## LACTATE ########

#lactate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_lac__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_lac__L_e_FWD-SPONT"] = (8.0832) / 100
carb_frac_min["RXN-EXCH_lac__L_e_FWD-SPONT"] = (6.7279) / 100

######## FORMATE ########

#formate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_for_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_for_e_FWD-SPONT"] = (22.1002) / 100
carb_frac_min["RXN-EXCH_for_e_FWD-SPONT"] = (19.2576) / 100

######## MALATE ########

#malate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_mal__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_mal__L_e_FWD-SPONT"] = (7.1676) / 100
carb_frac_min["RXN-EXCH_mal__L_e_FWD-SPONT"] = (0.0000010174) / 100

######## HYDROGEN GAS ########

#hydrogen gas export
#define bounds
#hydrogen has no carbons
carb_frac_rxns.append("RXN-EXCH_h2_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_h2_e_FWD-SPONT"] = (129.1105) / 100
carb_frac_min["RXN-EXCH_h2_e_FWD-SPONT"] = (118.9361) / 100

######## L-ALANINE ########

#L-alanine export
#define bounds
carb_frac_rxns.append("RXN-EXCH_ala__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_ala__L_e_FWD-SPONT"] = (1.2016) / 100
carb_frac_min["RXN-EXCH_ala__L_e_FWD-SPONT"] = (0.9946) / 100

######## L-VALINE ########

#L-valine export
#define bounds
carb_frac_rxns.append("RXN-EXCH_val__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_val__L_e_FWD-SPONT"] = (5.5585) / 100
carb_frac_min["RXN-EXCH_val__L_e_FWD-SPONT"] = (4.6205) / 100

######## L-ASPARTATE ########

#L-aspartate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_asp__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_asp__L_e_FWD-SPONT"] = (6.7393) / 100
carb_frac_min["RXN-EXCH_asp__L_e_FWD-SPONT"] = 0

######## L-THREONINE ########

#L-threonine export
#define bounds
carb_frac_rxns.append("RXN-EXCH_thr__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_thr__L_e_FWD-SPONT"] = (0.925) / 100
carb_frac_min["RXN-EXCH_thr__L_e_FWD-SPONT"] = 0

######## FUMARATE ########

#fumarate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_fum_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_fum_e_FWD-SPONT"] = (2.6678) / 100
carb_frac_min["RXN-EXCH_fum_e_FWD-SPONT"] = (0.000053212) / 100

######## L-LEUCINE ########

#L-leucine export
#define bounds
carb_frac_rxns.append("RXN-EXCH_leu__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_leu__L_e_FWD-SPONT"] = (1.193) / 100
carb_frac_min["RXN-EXCH_leu__L_e_FWD-SPONT"] = 0

######## L-GLUTAMATE ########

#L-glutamate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_glu__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_glu__L_e_FWD-SPONT"] = (2.3354) / 100
carb_frac_min["RXN-EXCH_glu__L_e_FWD-SPONT"] = 0

######## ALPHA-KETOGLUTERATE ########

#alpha-ketogluterate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_akg_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_akg_e_FWD-SPONT"] = (6.2633) / 100
carb_frac_min["RXN-EXCH_akg_e_FWD-SPONT"] = (0.00000088218) / 100

#turn off reverse reaction
carb_frac_rxns.append("RXN-EXCH_akg_e_REV-SPONT")
carb_frac_max["RXN-EXCH_akg_e_REV-SPONT"] = 0
carb_frac_min["RXN-EXCH_akg_e_REV-SPONT"] = 0

######## SUCCINATE ########

#succinate export
#define bounds
carb_frac_rxns.append("RXN-EXCH_succ_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_succ_e_FWD-SPONT"] = (10.5402) / 100
carb_frac_min["RXN-EXCH_succ_e_FWD-SPONT"] = 0

######## L-ASPARAGINE ########

#L-asparagine export
#define bounds
carb_frac_rxns.append("RXN-EXCH_asn__L_e_FWD-SPONT")
carb_frac_max["RXN-EXCH_asn__L_e_FWD-SPONT"] = (0.6049) / 100
carb_frac_min["RXN-EXCH_asn__L_e_FWD-SPONT"] = (0.4974) / 100

######## BIOMASS ########

#define bounds
#carb_frac_rxns.append("RXN-MFA_BIOMASS_FWD-SPONT")
#carb_frac_max["RXN-MFA_BIOMASS_FWD-SPONT"] = (8.8263 * 34.42066) / 100
#carb_frac_min["RXN-MFA_BIOMASS_FWD-SPONT"] = (8.0749 * 34.42066) / 100

#### Write the files for applying MFA bounds to the GAMS RBA model

In [13]:
#gams-formatted file for lower bounds of reactions from MFA
carb_min_frac_f = open('./'+def_out+'/carb_min_frac.txt', 'w')
carb_min_frac_f.write("/\n")

carb_max_frac_f = open('./'+def_out+'/carb_max_frac.txt', 'w')
carb_max_frac_f.write("/\n")

#gams-formatted subset of 
MFA_rxns_f = open('./'+def_out+'/carb_frac_rxns.txt',"w")
MFA_rxns_f.write("/\n")

for rxn in carb_frac_rxns:

  carb_min_frac_f.write("\'"+rxn+"\'\t"+str(carb_frac_min[rxn])+"\n")
  carb_max_frac_f.write("\'"+rxn+"\'\t"+str(carb_frac_max[rxn])+"\n")
  MFA_rxns_f.write("\'"+rxn+"\'\n")

carb_min_frac_f.write("/")
carb_max_frac_f.write("/")
MFA_rxns_f.write("/")

carb_min_frac_f.close()
carb_max_frac_f.close()
MFA_rxns_f.close()

In [18]:
sbml_file = './'+def_in+'/iCTH669_for_RBA.sbml'          #stoichiometric network file

model = cobra.io.read_sbml_model(sbml_file)

block_rxn_str = "/\n"

for rxn in model.reactions:

  if((rxn.lower_bound == 0) and (rxn.upper_bound == 0)):

    #then it is a blocke reaction, add to the blocked reaction list

    #first have to format for the RAM-style reactions, so need to get the approriate forward/reverse and enzyme tags
    if rxn.id in base_to_enz:

      for enz in base_to_enz[rxn.id]:

        rxn_str_fwd = "RXN-"+rxn.id+"_FWD-"+enz

        if (rxn_str_fwd in rxn_names): 

          block_rxn_str = block_rxn_str + "\'"+rxn_str_fwd+"\'\n"

        rxn_str_rev = "RXN-"+rxn.id+"_REV-"+enz

        if (rxn_str_rev in rxn_names): 

          block_rxn_str = block_rxn_str + "\'"+rxn_str_rev+"\'\n"

block_rxn_str = block_rxn_str + "/"

with open('./'+def_out+'/blocked_rxns.txt', 'w') as f:
    f.write(block_rxn_str)

#### Checks of specific values to ensure correct outputs

In [15]:
carb_frac_min['RXN-EXCH_co2_e_FWD-SPONT']

0.740563

In [16]:
carb_frac_max['RXN-EXCH_co2_e_FWD-SPONT']

0.7964140000000001

#### Copy model into folder with GAMS code

In [17]:
shutil.rmtree('../GAMS/'+def_out)
shutil.copytree('./'+def_out+'/',"../GAMS/"+def_out+"/")

'../GAMS/model_base/'